# Analyzing image texture using pyCERR


This notebook demonstrates how to filter images using pyCERR.

Supported filters include: *Mean*, *Sobel*, *Laplacian of Gaussian (LoG)*, *Laws'/Laws' Energy*, and *Gabor*.  
All filters are implemented as per the [IBSI2](https://theibsi.github.io/ibsi2/) standard, including support for 3D approximations and rotation invariance where defined.

## Install pyCERR (dev version)

In [2]:
%% capture
pip install git+https://github.com/cerr/pyCERR/

## Read sample DICOM images

* The dataset used here, comprising a Lung CT scan with tumor segmentation, was made available by Lambin et al. via CancerData and is included with pyCERR.

* Selected filter parameters used in compatibilty tests with the IBSI-2 standard are used in this demonstration.

In [3]:
import os, numpy as np
from cerr import datasets, plan_container as pc

# Path to DICOM data
sampleDataPath = os.path.dirname(datasets.__file__)
sampleDataDir = os.path.join(sampleDataPath, 'radiomics_phantom_dicom', 'PAT1')

# Import contents to planC
planC = pc.load_dcm_dir(sampleDataDir)

('PAT1', 'PAT1', '1.3.6.1.4.1.9590.100.1.2.96328687310426543129572151154132284399', '1.3.6.1.4.1.9590.100.1.2.258301620411152643708006163321128526885', 'RTSTRUCT', 'RTSTRUCT', 'RTSTRUCT', 'RTSTRUCT', 'RTSTRUCT', 'RTSTRUCT', 'RTSTRUCT')
('PAT1', 'PAT1', '1.3.6.1.4.1.9590.100.1.2.96328687310426543129572151154132284399', '1.3.6.1.4.1.9590.100.1.2.296658988911737913102339329841519593982', 'CT', 'CT', 'CT', 'CT', 'CT', 'CT', 'CT')


## Visualize input scan and tumor mask

In [4]:
from cerr import viewer as vwr

#Display scan and tumor mask
scanNum = 0
strNum = 0
doseNum = []

# Extract scan array
scan3M = planC.scan[scanNum].getScanArray()
scan3M = scan3M.astype(float)

# Display 
displayMode = '2d' # Set to '2d - path' or '3d - surface'
viewer, scan_layer, dose_layer, struct_layer = \
vwr.show_scan_struct_dose(scanNum, strNum, doseNum, planC, displayMode)

## Example-1 : Mean filter

*Note*: The examples below use the *Napari* package for visualization. *Napari* requires a display for rendering graphics and does not support visualization on a remote server (e.g. Google Colab).

### Review filter settings

In [5]:
import json, jsbeautifier
from cerr.radiomics import textureUtils 

options = jsbeautifier.default_options()
options.indent_size = 4

# Path to JSON settings file with filter parameters
settingsDir = os.path.join(sampleDataPath, 'radiomics_settings',\
                           'IBSIsettings','IBSI2Phase1')
# Read filter settings
settingsFile =  os.path.join(settingsDir,'IBSIPhase2-1ID1a1.json')
paramS, __ = textureUtils.loadSettingsFromFile(settingsFile)

# Display settings
print(jsbeautifier.beautify(json.dumps(paramS), options))

{
    "structures": ["wholeScan"],
    "imageType": {
        "Mean": {
            "KernelSize": [15, 15, 15]
        }
    },
    "settings": {
        "padding": [{
            "method": "padzeros",
            "size": [7, 7, 7]
        }]
    }
}


In [6]:
# Compute filter response
planC = textureUtils.generateTextureMapFromPlanC(planC, scanNum, strNum, settingsFile)
filtIdx = len(planC.scan)-1
strIdx = len(planC.structure)-1

In [7]:
#Display
scanNum = filtIdx
doseNum = []
strNum = 0
displayMode = '2d'     # Se to '2d - path' or '3d - surface'

viewer, scan_layer, dose_layer, struct_layer = \
vwr.show_scan_struct_dose(filtIdx, strIdx, doseNum, planC, displayMode)

## Example-2 : 3D LoG filter

In [8]:
settingsFile =  os.path.join(settingsDir,'IBSIPhase2-1ID2a.json')
planC = textureUtils.generateTextureMapFromPlanC(planC, scanNum, strNum, settingsFile)
print(len(planC.scan))

#Display 
filtIdx = len(planC.scan)-1
strIdx = len(planC.structure)-1
viewer, scan_layer, dose_layer, struct_layer = \
vwr.show_scan_struct_dose(filtIdx, strNum, doseNum, planC, displayMode)

## Example-3: Rotation-invariant Laws' energy filter

In [9]:
# Read filter settings
settingsFile =  os.path.join(settingsDir,'IBSIPhase2-1ID3c3.json')
planC = textureUtils.generateTextureMapFromPlanC(planC, scanNum, strNum, settingsFile)

#Display 
filtIdx = len(planC.scan)-1
strIdx = len(planC.structure)-1
viewer, scan_layer, dose_layer, struct_layer = \
vwr.show_scan_struct_dose(filtIdx, strNum, doseNum, planC, displayMode)

## Example-4: 2D Gabor filter

In [10]:
settingsFile =  os.path.join(settingsDir,'IBSIPhase2-1ID4a1.json')
planC = textureUtils.generateTextureMapFromPlanC(planC, scanNum, strNum, settingsFile)

#Display 
filtIdx = len(planC.scan)-1
strIdx = len(planC.structure)-1
viewer, scan_layer, dose_layer, struct_layer = \
vwr.show_scan_struct_dose(filtIdx, strNum, doseNum, planC, displayMode)